# Onderzoeksvraag 2

### Doel
De tweede onderzoeksvraag die we willen beantwoorden luidt als volgt:

**In hoeverre kan de IMDB-score van een film worden voorspeld op basis van de sociale media-populariteit van de cast (zoals Facebook-likes) en de bechdeltest rating uit de externe dataset?**  
-De targetvariabele is de IMDB-score van de film  
-Featurevariabelen zijn Facebook-likes, en of de bechdeltest geslaagd is voor die film

### Modelkeuze

DecisiontreeClassifier kan niet, omdat de voorspellen waarde niet categorisch is maar numeriek

Regressie lijkt de beste optie te zijn

Todo

-importeren gecleande data  
-importeren en koppelen externe dataset  
-baseline model maken  
-hyperparameters aanpassen  
-conclusies en beantwoorden onderzoeksvraag
